In [ ]:
import optiver2023
import xgboost as xgb
import pandas as pd
from sklearn.metrics import mean_absolute_error

env = optiver2023.make_env()
iter_test = env.iter_test()

In [ ]:
train_df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
train_df

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [ ]:
train_df['imbalance_buy_sell_flag'].unique()

array([ 1, -1,  0])

In [ ]:
train_df = train_df.drop(['stock_id', 'row_id', 'time_id'], axis=1)

In [ ]:
missing = {}
for col_name in train_df.columns:
    print ("column:",col_name,".Missing:",sum(train_df[col_name].isnull()))
    if sum(train_df[col_name].isnull()) > 0:
        missing[col_name] = sum(train_df[col_name].isnull())

column: date_id .Missing: 0
column: seconds_in_bucket .Missing: 0
column: imbalance_size .Missing: 220
column: imbalance_buy_sell_flag .Missing: 0
column: reference_price .Missing: 220
column: matched_size .Missing: 220
column: far_price .Missing: 2894342
column: near_price .Missing: 2857180
column: bid_price .Missing: 220
column: bid_size .Missing: 0
column: ask_price .Missing: 220
column: ask_size .Missing: 0
column: wap .Missing: 220
column: target .Missing: 88


In [ ]:
for ind in missing:
    train_df[ind].fillna(train_df[ind].mean(), inplace=True)

In [ ]:
#checking if fillana worked
for col_name in train_df.columns:
    print ("column:",col_name,".Missing:",sum(train_df[col_name].isnull()))

column: date_id .Missing: 0
column: seconds_in_bucket .Missing: 0
column: imbalance_size .Missing: 0
column: imbalance_buy_sell_flag .Missing: 0
column: reference_price .Missing: 0
column: matched_size .Missing: 0
column: far_price .Missing: 0
column: near_price .Missing: 0
column: bid_price .Missing: 0
column: bid_size .Missing: 0
column: ask_price .Missing: 0
column: ask_size .Missing: 0
column: wap .Missing: 0
column: target .Missing: 0


In [ ]:
ytr = train_df['target']
xtr = train_df.drop(['target'], axis=1)
print(xtr.head())

   date_id  seconds_in_bucket  imbalance_size  imbalance_buy_sell_flag  \
0        0                  0      3180602.69                        1   
1        0                  0       166603.91                       -1   
2        0                  0       302879.87                       -1   
3        0                  0     11917682.27                       -1   
4        0                  0       447549.96                       -1   

   reference_price  matched_size  far_price  near_price  bid_price  bid_size  \
0         0.999812   13380276.64   1.001713     0.99966   0.999812  60651.50   
1         0.999896    1642214.25   1.001713     0.99966   0.999896   3233.04   
2         0.999561    1819368.03   1.001713     0.99966   0.999403  37956.00   
3         1.000171   18389745.62   1.001713     0.99966   0.999999   2324.90   
4         0.999532   17860614.95   1.001713     0.99966   0.999394  16485.54   

   ask_price   ask_size  wap  
0   1.000026    8493.03  1.0  
1   1.000660

In [ ]:
model = xgb.XGBRegressor(tree_method="hist", eval_metric=mean_absolute_error, random_state=42)
model.fit(xtr, ytr)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False,
             eval_metric=<function mean_absolute_error at 0x7ae4666d5120>,
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [ ]:
counter = 0
for (test_df, revealed_targets, sample_prediction) in iter_test:
    #pre-processing test data
    test_df = test_df.drop(['stock_id', 'row_id'], axis=1)
    missing = {}
    for col_name in test_df.columns:
        if sum(test_df[col_name].isnull()) > 0:
            missing[col_name] = sum(test_df[col_name].isnull())

    for ind in missing:
        test_df[ind].fillna(test_df[ind].mean(), inplace=True)

    # predictions
    sample_prediction['target'] = model.predict(test_df)

    env.predict(sample_prediction)
    counter += 1


This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
